In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('parch-and-posey.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,92,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,93,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,94,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,221,"CREATE TABLE accounts (\tid integer,\tname bpc..."


# Subqueries Part II

In the first subquery you wrote, you created a table that you could then query again in the FROM statement. However, if you are only returning a single value, you might use that value in a logical statement like WHERE, HAVING, or even SELECT - the value could be nested within a CASE statement.

On the next concept, we will work through this example, and then you will get some practice on answering some questions on your own.

Expert Tip
Note that you should not include an alias when you write a subquery in a conditional statement. This is because the subquery is treated as an individual value (or set of values in the IN case) rather than as a table.

Also, notice the query here compared a single value. If we returned an entire column IN would need to be used to perform a logical argument. If we are returning an entire table, then we must use an ALIAS for the table, and perform additional logic on the entire table.

In [8]:
pd.read_sql_query(sql='''
SELECT *
FROM orders
WHERE strftime('%Y-%m', occurred_at) = 
(SELECT MIN(strftime('%Y-%m', occurred_at)) AS min_month
FROM orders)
ORDER BY occurred_at;
''', con=conn)

,id,account_id,occurred_at,standard_qty,gloss_qty,poster_qty,total,standard_amt_usd,gloss_amt_usd,poster_amt_usd,total_amt_usd
0,5786,2861,2013-12-04 04:22:44,0,48,33,81,0.00,359.52,267.96,627.48
1,2415,2861,2013-12-04 04:45:54,490,15,11,516,2445.10,112.35,89.32,2646.77
2,4108,4311,2013-12-04 04:53:25,528,10,0,538,2634.72,74.90,0.00,2709.62
3,4489,1281,2013-12-05 20:29:16,0,37,0,37,0.00,277.13,0.00,277.13
4,287,1281,2013-12-05 20:33:56,492,73,0,565,2455.08,546.77,0.00,3001.85
...,...,...,...,...,...,...,...,...,...,...,...
94,4756,1521,2013-12-31 00:18:22,10,60,29,99,49.90,449.40,235.48,734.78
95,702,1521,2013-12-31 00:36:59,266,0,26,292,1327.34,0.00,211.12,1538.46
96,3057,3371,2013-12-31 02:04:00,79,75,14,168,394.21,561.75,113.68,1069.64
97,2764,3101,2013-12-31 12:58:47,119,36,0,155,593.81,269.64,0.00,863.45


In [10]:
# subquery

pd.read_sql_query(sql='''
SELECT *
FROM (SELECT MIN(strftime('%Y-%m', occurred_at)) AS min_month
FROM orders);
''', con=conn)

,min_month
0,2013-12
